# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the dataset with `mongoimport --db uk_food --collection establishments --file Resources/establishments.json --jsonArray`

In [66]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [67]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [68]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'fruits_db', 'gardenDB', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [69]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [70]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [71]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('65ab0c47ad263667f4260291'), 'FHRSID': 1290215, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000083029', 'BusinessName': 'FirstLight Bar & Café', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': '1 The Droveway', 'AddressLine2': 'St Margarets Bay', 'AddressLine3': 'Kent', 'AddressLine4': '', 'PostCode': 'CT15 6DH', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2020-11-09T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.378231', 'latitude': '51.150625'}, 'RightToReply': '', 'Distance': 4587.64041272672, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'totalCount'

In [72]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [74]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [75]:
# Insert the new restaurant into the collection
insert_result = establishments.insert_one(new_restaurant)

In [76]:
# Check that the new restaurant was inserted
if insert_result.inserted_id:
    print("Restaurant added successfully with ID:", insert_result.inserted_id)
else:
    print("Failed to add the restaurant.")

Restaurant added successfully with ID: 65ab0c69f23ea016185d1f12


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [77]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}

projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}

result = establishments.find_one(query, projection)

if result:
    print("BusinessTypeID:", result["BusinessTypeID"])
    print("BusinessType:", result["BusinessType"])
else:
    print("Business type not found.")

BusinessTypeID: 1
BusinessType: Restaurant/Cafe/Canteen


3. Update the new restaurant with the `BusinessTypeID` you found.

In [78]:
# Update the new restaurant with the correct BusinessTypeID
query = {"BusinessName": "Penang Flavours"}

update_operation = {"$set": {"BusinessTypeID": "1"}}

update_result = establishments.update_one(query, update_operation)


In [79]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("Restaurant updated successfully.")
else:
    print("Restaurant not found or update failed.")

Restaurant updated successfully.


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [80]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

count = establishments.count_documents(query)

print(f"Number of documents with LocalAuthorityName 'Dover': {count}")

Number of documents with LocalAuthorityName 'Dover': 994


In [81]:
# Delete all documents where LocalAuthorityName is "Dover"
query = {"LocalAuthorityName": "Dover"}

delete_result = establishments.delete_many(query)


In [82]:
# Check if any remaining documents include Dover
print(f"Number of documents deleted: {delete_result.deleted_count}")


Number of documents deleted: 994


In [83]:
# Check that other documents remain with 'find_one'
sample_query = {}

result = establishments.find_one(sample_query)

if result:
    print("At least one document remains in the collection.")
else:
    print("No documents found in the collection.")

At least one document remains in the collection.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [89]:
query = {
    "$or": [
        {"geocode.latitude": {"$type": "string"}},
        {"geocode.longitude": {"$type": "string"}}
    ]
}

for doc in establishments.find(query):
    latitude_float = float(doc['geocode']['latitude'])
    longitude_float = float(doc['geocode']['longitude'])
    
    update_operation = {
        "$set": {
            "geocode.latitude": latitude_float,
            "geocode.longitude": longitude_float
        }
    }
    
    establishments.update_many({"_id": doc["_id"]}, update_operation)

print("Latitude and longitude fields converted to float.")

Latitude and longitude fields converted to float.


Use `update_many` to convert `RatingValue` to integer numbers.

In [85]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

UpdateResult({'n': 4091, 'nModified': 4091, 'ok': 1.0, 'updatedExisting': True}, acknowledged=True)

In [86]:
integer_type_check = establishments.find_one({"RatingValue": {"$type": "int"}})

# Check if "RatingValue" contains valid integer values in documents
invalid_integer_check = establishments.find_one({"RatingValue": {"$type": "string", "$not": {"$regex": "^[0-9]+$"}}})

# Check the results of the validations
if integer_type_check:
    print("The 'RatingValue' field is already of integer type in some documents.")
elif invalid_integer_check:
    print("Some documents contain non-integer values in the 'RatingValue' field.")
else:
    print("The 'RatingValue' field is suitable for conversion to integers.")

The 'RatingValue' field is suitable for conversion to integers.


In [92]:
# Change the data type from String to Integer for RatingValue
query = {
    "RatingValue": {"$type": "string"}
}

for doc in establishments.find(query):
    rating_integer = int(doc['RatingValue'])
    
    update_operation = {
        "$set": {"RatingValue": rating_integer
        }
    }
    
    
    establishments.update_many({"_id": doc["_id"]}, update_operation)


In [93]:
# Check that the coordinates and rating value are now numbers
sample_query = {}

# Find one document using the sample query
sample_document = establishments.find_one(sample_query)

# Check the data types of latitude, longitude, and RatingValue in the sample document
latitude_type = type(sample_document['geocode']['latitude'])
longitude_type = type(sample_document['geocode']['longitude'])
rating_value_type = type(sample_document['RatingValue'])

print("Data types:")
print(f"Latitude: {latitude_type}")
print(f"Longitude: {longitude_type}")
print(f"RatingValue: {rating_value_type}")

Data types:
Latitude: <class 'float'>
Longitude: <class 'float'>
RatingValue: <class 'int'>
